# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kailua-kona,19.6406,-155.9956,26.98,77,40,3.09,US,1706844624
1,1,thompson,55.7435,-97.8558,-8.91,92,100,3.60,CA,1706844911
2,2,georgetown,5.4112,100.3354,30.96,70,20,3.60,MY,1706844622
3,3,remire-montjoly,4.9167,-52.2667,23.02,100,100,0.00,GF,1706844902
4,4,port lincoln,-34.7333,135.8667,21.44,66,45,4.35,AU,1706844911


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
humidity_level = city_data_df["Humidity"]

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 900, 
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = .75
)


# Display the map
map_plot_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"]< 27) & (city_data_df["Max Temp"] > 21)]

# #Wind speed less than 4.5 m/s
ideal_wind_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"]< 4.5]

# #Zero cloudiness
ideal_no_clouds_df = ideal_wind_df.loc[ideal_wind_df["Cloudiness"] == 0]

# # Drop any rows with null values
ideal_no_clouds_df = ideal_no_clouds_df.dropna(how = "any")

# # Display sample data
ideal_no_clouds_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,mermoz boabab,14.7065,-17.4758,26.31,68,0,4.12,SN,1706844916
207,207,pisco,-13.7000,-76.2167,24.03,78,0,3.60,PE,1706844857
380,380,ribas do rio pardo,-20.4431,-53.7592,24.83,51,0,2.15,BR,1706844980
424,424,mar del plata,-38.0023,-57.5575,24.93,69,0,3.13,AR,1706844722
541,541,vallenar,-28.5708,-70.7581,21.44,63,0,0.77,CL,1706844754


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kailua-kona,US,19.6406,-155.9956,77,
1,thompson,CA,55.7435,-97.8558,92,
2,georgetown,MY,5.4112,100.3354,70,
3,remire-montjoly,GF,4.9167,-52.2667,100,
4,port lincoln,AU,-34.7333,135.8667,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { "categories":"accommodation.hotel", "apiKey":geoapify_key }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kailua-kona - nearest hotel: Kona Seaside Hotel
thompson - nearest hotel: Thompson Inn
georgetown - nearest hotel: Page 63 hostel
remire-montjoly - nearest hotel: Complexe Belova
port lincoln - nearest hotel: Boston Hotel
chorbane - nearest hotel: No hotel found
belgrade - nearest hotel: Хилтон Београд
iskateley - nearest hotel: Авантаж
altay - nearest hotel: Sunshine Holiday Hotel
waitangi - nearest hotel: Hotel Chathams
hamilton - nearest hotel: North Vista Manor
pandan - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
edinburgh of the seven seas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
atafu village - nearest hotel: No hotel found
correntina - nearest hotel: Pousada Silva
tobelo - nearest hotel: JJ Guesthouse
ardestan - nearest hotel: Tourism Hotel
olonkinbyen - nearest hotel: No hotel found
ampanihy - nearest hotel: Hotel restaurant Angora
margaret river - near

kavieng - nearest hotel: Kavieng Hotel
guixi - nearest hotel: No hotel found
hola - nearest hotel: No hotel found
talnakh - nearest hotel: Талнах
opuwo - nearest hotel: Ohakane Lodge
guiberoua - nearest hotel: No hotel found
bemidji - nearest hotel: Hampton Inn and Suites
katima mulilo - nearest hotel: Zambezi River Lodge
as-suwayda - nearest hotel: فندق ديونزياس
reuleuet - nearest hotel: No hotel found
altai - nearest hotel: Altai hotel 40000-100000T
nchelenge - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
sao paulo de olivenca - nearest hotel: No hotel found
ippy - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
pisco - nearest hotel: La Portada
axim - nearest hotel: Axim Beach Hotel
fisterra - nearest hotel: Hotel Temático do Banco Azul
mil'kovo - nearest hotel: Дрозды
pergamos - nearest hotel: Ocean Bay
sines - nearest hotel: Sinerama
touros - nearest hotel: Pousada Atlântico
north

santa elena de uairen - nearest hotel: Posada Villa Fairmont
shakawe - nearest hotel: EdenWood guesthouse
ribas do rio pardo - nearest hotel: Hotel São Francisco
bel ombre - nearest hotel: Villa La Cachette
boulder city - nearest hotel: Boulder Dam Hotel
wakkanai - nearest hotel: ホテル喜登
siwa oasis - nearest hotel: فندق الكيلانى
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
weno - nearest hotel: High Tide Hotel
buritizeiro - nearest hotel: Hotel Pousada Sertão Veredas
idri - nearest hotel: No hotel found
zuenoula - nearest hotel: Hotel le Prestige
mount pearl - nearest hotel: Sandman Signature St. John's
kuta - nearest hotel: Ida Hotel Bali
malanje - nearest hotel: Hotel PORTVGALIA
kalmunai - nearest hotel: Saji new food
naze - nearest hotel: ビジネス旅館畠山
amla - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
kapa'a - nearest hotel: Pono Kai Resort
puerto aysen - nearest hotel: Hostal Dendal
lashio - nearest hotel: Golden Kaniri Hotel
inongo - nearest hotel: Chez LELE IYOLO


leava - nearest hotel: Fia fia
okhotsk - nearest hotel: No hotel found
mhamid - nearest hotel: Hotel Kasbah Azalay
arteche - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,kailua-kona,US,19.6406,-155.9956,77,Kona Seaside Hotel
1,thompson,CA,55.7435,-97.8558,92,Thompson Inn
2,georgetown,MY,5.4112,100.3354,70,Page 63 hostel
3,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
4,port lincoln,AU,-34.7333,135.8667,66,Boston Hotel
...,...,...,...,...,...,...
567,manado,ID,1.4870,124.8455,74,Swiss Belhotel
568,leava,WF,-14.3000,-178.1667,79,Fia fia
569,okhotsk,RU,59.3833,143.3000,75,No hotel found
570,mhamid,MA,29.8200,-5.7200,28,Hotel Kasbah Azalay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
#%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map plot
map_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)